In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Deep Learning Challenge

In [ ]:
from google.colab import files

!rm -rf models.py
!rm -rf visualization.py
!rm -rf data_preparation.py
files.upload()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from data_preparation import load_data, delete_outliers
import numpy as np
from visualization import plot_history


X, y = load_data()

X, y, num_outliers = delete_outliers(X, y)

# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

# Further split train_val into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, stratify=y_train_val)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

print(np.unique(y_train, return_counts=True))
print(np.unique(y_val, return_counts=True))

In [ ]:
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.metrics import Precision, Recall


def build_transfer_model(input_shape, name="TransferModel"):
    mobile = tfk.applications.MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights="imagenet",
        pooling='avg',
    )
    
    # Use the supernet as feature extractor, i.e. freeze all its weigths
    mobile.trainable = False

    # Create an input layer with shape (224, 224, 3)
    inputs = tfk.Input(shape=input_shape)

    # Connect MobileNetV2 to the input
    x = mobile(inputs)

    x = tfkl.Dense(256, activation="relu")
    
    x = tfkl.Dense(64, activation="relu")

    # Add a Dense layer with 2 units and softmax activation as the classifier
    outputs = tfkl.Dense(1, activation='sigmoid')(x)

    # Create a Model connecting input and output
    tl_model = tfk.Model(inputs=inputs, outputs=outputs, name=name)

    # Compile the model with Categorical Cross-Entropy loss and Adam optimizer
    tl_model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy', Precision(name="precision"), Recall(name="recall")])

    return tl_model, mobile


In [ ]:
model, base_model = build_transfer_model(X_train.shape[1:])

history = model.fit(
    x = preprocess_input(X_train * 255),
    y = y_train,
    batch_size = 32,
    epochs = 200,
    validation_data = (preprocess_input(X_val * 255), y_val),
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history


plot_history(history, model.name)

In [ ]:
for layer in base_model.layers[-12]:
    layer.trainable = True

model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy', Precision(name="precision"), Recall(name="recall")])


history = model.fit(
    x = preprocess_input(X_train * 255),
    y = y_train,
    batch_size = 32,
    epochs = 200,
    validation_data = (preprocess_input(X_val * 255), y_val),
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history


plot_history(history, model.name)